In [1]:
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [2]:
import pyLDAvis, pandas as pd, numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

//anaconda/lib/python3.5/site-packages/IPython/core/formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
//anaconda/lib/python3.5/site-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
//anaconda/lib/python3.5/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
//anaconda/lib/python3.5/site-packages/IPython/core/formatters.py:672: DeprecationWarning: PlainTextFormatter._type_printers_default is deprecated: use @default decorator instead.
  def _type_printers_default(self):


In [11]:
frames = []
for i in range(9):
    newf = pd.read_json("Data/2016/opcomments2016-0" + str(1+i), encoding="utf-8")
    frames.append(newf)
for i in range(3):
    newf = pd.read_json("Data/2016/opcomments2016-" + str(10+i), encoding="utf-8")
    frames.append(newf)

df = pd.concat(frames)
df = df.reset_index(drop=True)
df["date"]= pd.to_datetime(df['created_utc'],unit='s')

In [41]:
df.columns

Index(['author', 'author_flair_css_class', 'author_flair_text', 'body',
       'controversiality', 'created_utc', 'distinguished', 'edited', 'gilded',
       'id', 'link_id', 'parent_id', 'retrieved_on', 'score', 'stickied',
       'subreddit', 'subreddit_id', 'ups', 'date'],
      dtype='object')

In [19]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=2000, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(df["body"])

In [23]:
%%time
from empath import Empath
lexicon = Empath()
empath_feat = [lexicon.analyze(post) for post in df['body'].loc[:1000]]

CPU times: user 13.6 s, sys: 267 ms, total: 13.9 s
Wall time: 14.3 s


In [24]:
%%time
lda = LatentDirichletAllocation(n_components=10, random_state=0)
lda.fit(tfidf)

//anaconda/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:532: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


CPU times: user 4min 50s, sys: 1.44 s, total: 4min 52s
Wall time: 4min 58s


In [38]:
#'opioid' in 'opioidssssdfasdfas'
'opioidssssdfasdfas'.find('opioid')

0

In [34]:
opioid, fentanyl, fent, heroin, cocaine, codeine, morphine, methadone, opium, heroin, suboxone

{0: ['fentanyl',
  'fent',
  'heroin',
  'people',
  'drug',
  'drugs',
  'cut',
  'dealers',
  'potent',
  'like',
  'od',
  'cocaine',
  'gt',
  'just',
  'laced',
  'legal',
  'analogues',
  'don',
  'https',
  'dangerous'],
 1: ['percs',
  'codeine',
  'crazy',
  'piece',
  'like',
  'love',
  'nice',
  'pst',
  'reddit',
  'oxy',
  'song',
  'looks',
  'im',
  'fent',
  'future',
  'really',
  'watch',
  'glass',
  'mixing',
  'good'],
 2: ['oxy',
  'lol',
  'tho',
  'pressed',
  'xanax',
  'pills',
  '30mg',
  '10mg',
  'lean',
  'like',
  '30',
  'pill',
  'bars',
  'hydro',
  'fake',
  'dillies',
  'morphine',
  'just',
  'hydros',
  'codeine'],
 3: ['morphine',
  'methadone',
  'clinic',
  'poppy',
  'seeds',
  'tea',
  'test',
  'clinics',
  'juice',
  'nah',
  'opium',
  'heroin',
  'does',
  'seed',
  'fent',
  'drip',
  'suboxone',
  'fuck',
  'yeah',
  'positive'],
 4: ['pain',
  'morphine',
  'just',
  'like',
  'got',
  'time',
  'fentanyl',
  'hospital',
  'didn',
  'g

In [33]:
vocab = tfidf_vectorizer.get_feature_names()

n_top_words = 20
k = 2
topic_words = {}

for topic, comp in enumerate(lda.components_):
    # for the n-dimensional array "arr":
    # argsort() returns a ranked n-dimensional array of arr, call it "ranked_array"
    # which contains the indices that would sort arr in a descending fashion
    # for the ith element in ranked_array, ranked_array[i] represents the index of the
    # element in arr that should be at the ith index in ranked_array
    # ex. arr = [3,7,1,0,3,6]
    # np.argsort(arr) -> [3, 2, 0, 4, 5, 1]
    # word_idx contains the indices in "topic" of the top num_top_words most relevant
    # to a given topic ... it is sorted ascending to begin with and then reversed (desc. now)    
    word_idx = np.argsort(comp)[::-1][:n_top_words]

    # store the words most relevant to the topic
    topic_words[topic] = [vocab[i] for i in word_idx]

for topic, words in topic_words.items():
    print('Topic: %d' % topic)
    print('  %s' % ', '.join(words))


Topic: 0
  fentanyl, fent, heroin, people, drug, drugs, cut, dealers, potent, like, od, cocaine, gt, just, laced, legal, analogues, don, https, dangerous
Topic: 1
  percs, codeine, crazy, piece, like, love, nice, pst, reddit, oxy, song, looks, im, fent, future, really, watch, glass, mixing, good
Topic: 2
  oxy, lol, tho, pressed, xanax, pills, 30mg, 10mg, lean, like, 30, pill, bars, hydro, fake, dillies, morphine, just, hydros, codeine
Topic: 3
  morphine, methadone, clinic, poppy, seeds, tea, test, clinics, juice, nah, opium, heroin, does, seed, fent, drip, suboxone, fuck, yeah, positive
Topic: 4
  pain, morphine, just, like, got, time, fentanyl, hospital, didn, gt, surgery, don, doctor, did, ve, said, people, gave, really, know
Topic: 5
  just, like, methadone, oxy, don, ve, people, day, high, really, know, time, good, fent, feel, kratom, heroin, opiates, think, use
Topic: 6
  morphine, hydromorphone, opioid, heroin, schedule, gt, synthetic, kratom, opium, maintenance, opioids, canna

In [26]:
pyLDAvis.sklearn.prepare(lda, tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
5      41.482761        1       1  0.149108  0.003505
4      23.914572        1       2  0.134455  0.017777
0      11.511882        1       3  0.132107  0.052716
2       4.355135        1       4  0.112492 -0.092515
7       3.913669        1       5  0.040764 -0.121872
6       3.585144        1       6  0.039513  0.139070
1       3.477706        1       7  0.048193 -0.104195
3       2.715073        1       8  0.021868  0.113244
9       2.524986        1       9 -0.339379  0.282446
8       2.519070        1      10 -0.339120 -0.290176, topic_info=     Category         Freq          Term        Total  loglift  logprob
term                                                                   
356   Default  5575.000000       codeine  5575.000000  30.0000  30.0000
1141  Default  5338.000000      morphine  5338.000000  29.0000  29.0000
1102  Default  3556.000000     methadone  3556.000000  28.0000  28.0000
1159  Default  2526.000000         narco  2526.000000  27.0000  27.0000
1260  Default  4240.000000           oxy  4240.000000  26.0000  26.0000
1758  Default  1553.000000         syrup  1553.000000  25.0000  25.0000
1261  Default  2050.000000     oxycodone  2050.000000  24.0000  24.0000
670   Default  4177.000000      fentanyl  4177.000000  23.0000  23.0000
414   Default  1202.000000         cough  1202.000000  22.0000  22.0000
1300  Default  1521.000000         percs  1521.000000  21.0000  21.0000
1192  Default  1304.000000         norco  1304.000000  20.0000  20.0000
350   Default  1187.000000        clinic  1187.000000  19.0000  19.0000
858   Default  1610.000000   hydrocodone  1610.000000  18.0000  18.0000
816   Default  2924.000000        heroin  2924.000000  17.0000  17.0000
1664  Default  1056.000000     spaceedge  1056.000000  16.0000  16.0000
669   Default  3035.000000          fent  3035.000000  15.0000  15.0000
1142  Default   964.000000        morpho   964.000000  14.0000  14.0000
1405  Default   950.000000  promethazine   950.000000  13.0000  13.0000
1005  Default  3879.000000          like  3879.000000  12.0000  12.0000
1427  Default   820.000000           que   820.000000  11.0000  11.0000
958   Default  1257.000000        kratom  1257.000000  10.0000  10.0000
240   Default   780.000000          blue   780.000000   9.0000   9.0000
959   Default   692.000000            la   692.000000   8.0000   8.0000
907   Default   653.000000            ir   653.000000   7.0000   7.0000
583   Default   623.000000            el   623.000000   6.0000   6.0000
1348  Default   634.000000         poppy   634.000000   5.0000   5.0000
1784  Default   757.000000          test   757.000000   4.0000   4.0000
1269  Default  2602.000000          pain  2602.000000   3.0000   3.0000
1324  Default  1305.000000         pills  1305.000000   2.0000   2.0000
761   Default  1386.000000            gt  1386.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
140   Topic10    43.749307        andrew    45.123245   3.6504  -5.7518
1942  Topic10    41.269129       wellity    42.642412   3.6485  -5.8102
933   Topic10    28.817388       keyword    30.190709   3.6347  -6.1693
52    Topic10    27.608076        52x17w    28.981359   3.6327  -6.2122
702   Topic10    27.608076    freestinks    28.981359   3.6327  -6.2122
450   Topic10    27.608076       d7opfnn    28.981359   3.6327  -6.2122
471   Topic10    27.608076         deary    28.981359   3.6327  -6.2122
919   Topic10    27.608076     joerogan2    28.981359   3.6327  -6.2122
29    Topic10    23.396105          20by    24.769389   3.6242  -6.3777
55    Topic10    19.473052          60by    20.846336   3.6131  -6.5612
1544  Topic10   191.543623          sale   205.099046   3.6129  -4.2752
720   Topic10    19.197211      gargoyle    20.570497   3.6122  -6.5755
1472  Topic10    15.531550       reddit2    16.904833   3.5966  -6.78

In [28]:
%%time
lda = LatentDirichletAllocation(n_components=20, random_state=0)
lda.fit(tfidf)

CPU times: user 4min 24s, sys: 1.06 s, total: 4min 25s
Wall time: 4min 11s


In [29]:
pyLDAvis.sklearn.prepare(lda, tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
12     50.683880        1       1  0.150665 -0.000892
10     12.337640        1       2  0.184248  0.039416
5       3.911300        1       3  0.207133  0.029503
1       3.502249        1       4  0.161292  0.083946
9       3.498929        1       5  0.164228 -0.115706
8       2.229140        1       6  0.050880  0.031653
11      2.099957        1       7  0.088505 -0.139451
3       2.042793        1       8 -0.198292 -0.118523
19      1.984580        1       9  0.076770  0.166865
15      1.921919        1      10 -0.311452 -0.099900
13      1.850674        1      11  0.082063 -0.191479
6       1.808453        1      12  0.044175  0.034158
0       1.699755        1      13 -0.269565 -0.058171
4       1.658454        1      14  0.102153 -0.055392
18      1.650254        1      15  0.073417 -0.001918
7       1.604895        1      16 -0.125007  0.213755
17      1.438422        1      17 -0.097551 -0.188419
16      1.389090        1      18 -0.134034  0.173501
14      1.368090        1      19 -0.144694  0.198090
2       1.319525        1      20 -0.104933 -0.001038, topic_info=     Category         Freq         Term        Total  loglift  logprob
term                                                                  
356   Default  4393.000000      codeine  4393.000000  30.0000  30.0000
670   Default  4855.000000     fentanyl  4855.000000  29.0000  29.0000
1141  Default  4619.000000     morphine  4619.000000  28.0000  28.0000
1261  Default  2390.000000    oxycodone  2390.000000  27.0000  27.0000
1102  Default  2987.000000    methadone  2987.000000  26.0000  26.0000
1300  Default  1769.000000        percs  1769.000000  25.0000  25.0000
1260  Default  4385.000000          oxy  4385.000000  24.0000  24.0000
816   Default  2992.000000       heroin  2992.000000  23.0000  23.0000
858   Default  1919.000000  hydrocodone  1919.000000  22.0000  22.0000
1269  Default  2639.000000         pain  2639.000000  21.0000  21.0000
669   Default  3254.000000         fent  3254.000000  20.0000  20.0000
1159  Default  1347.000000        narco  1347.000000  19.0000  19.0000
1192  Default  1300.000000        norco  1300.000000  18.0000  18.0000
761   Default  1351.000000           gt  1351.000000  17.0000  17.0000
1758  Default   912.000000        syrup   912.000000  16.0000  16.0000
1005  Default  3797.000000         like  3797.000000  15.0000  15.0000
1142  Default   831.000000       morpho   831.000000  14.0000  14.0000
1107  Default  1080.000000           mg  1080.000000  13.0000  13.0000
1028  Default  1056.000000          lol  1056.000000  12.0000  12.0000
350   Default   858.000000       clinic   858.000000  11.0000  11.0000
362   Default  1006.000000          com  1006.000000  10.0000  10.0000
240   Default   749.000000         blue   749.000000   9.0000   9.0000
1297  Default  2677.000000       people  2677.000000   8.0000   8.0000
1185  Default   823.000000         nice   823.000000   7.0000   7.0000
1861  Default   811.000000      tylenol   811.000000   6.0000   6.0000
555   Default  1805.000000         drug  1805.000000   5.0000   5.0000
707   Default  1119.000000         fuck  1119.000000   4.0000   4.0000
429   Default   768.000000        crazy   768.000000   3.0000   3.0000
414   Default   707.000000        cough   707.000000   2.0000   2.0000
1042  Default   873.000000         love   873.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
594   Topic20    59.604504    enjoyable   100.986851   3.8006  -4.7959
609   Topic20    75.063120    everybody   130.806005   3.7725  -4.5653
1890  Topic20    94.485778        usual   169.451696   3.7438  -4.3352
416   Topic20    92.342119        count   169.836964   3.7186  -4.3582
1366  Topic20   144.797664        power   272.477303   3.6957  -3.9083
1973  Topic20   143.249036        worry   271.536429   3.6884  -3.9191
